In [1]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import csv
import os
import requests
import re

In [2]:
def getClassification():################获取全部图书分类信息
        resp=urlopen('http://category.dangdang.com/?ref=www-0-C')
        soup=BeautifulSoup(resp,'html.parser')
        ##############目录
        General_classification =soup.find('div',id='floor_1')
    
        Gclass,Sclass,sclassion=[],[],[]###总目录和分目录获取
        for i in General_classification:
            try:
                Gclass.append(i.a.string.encode('iso-8859-1').decode('gb2312'))####需要编码解码消除乱码
                Sclass.append(i.ul)
            except:
                Gclass.append('港台图书')###特殊处理
                Sclass.append(i.ul)  
                
        del Gclass[0];del Gclass[-1];del Sclass[0];del Gclass[-1] #####删除头尾无用部分  53-->51个数
        
        sclassion,sclassurl,lensclass=[],[],[] #####分目录名，url,每个分类的长度，调用时sclassion[x:x+1]
        for i in range(len(Gclass)):
            lensc=0    ###计数每个分类的个数
            for j in Sclass[i]:
                if j.a.string.encode('iso-8859-1').decode('gb2312')=='更多':
                     continue
                sclassion.append(j.a.string.encode('iso-8859-1').decode('gb2312'))
                sclassurl.append(j.a.get('href'))
                lensc+=1
            lensclass.append(lensc)        
            
        return Gclass,sclassion,sclassurl,lensclass

In [3]:
Gclass,sclassion,sclassurl,lensclass=getClassification()#####50 679 679 50  ######length

In [4]:
print(
    len(Gclass),
    len(sclassion),
    len(sclassurl),
    len(lensclass)
)

50 679 679 50


In [12]:
pivot=0
for i in range(len(Gclass)):
    #print(Gclass[i],sclassion[lensclass[i]:lensclass[i+1],sclassion[lensclass[i]:lensclass[i+1])
    if Gclass[i]=='韩文原版书':
        continue
    print(Gclass[i])
    print(sclassion[pivot:pivot+lensclass[i]])
    print(sclassurl[pivot:pivot+lensclass[i]])
    pivot+=lensclass[i]

青春文学
['影视写真', '穿越/重生/架空', '叛逆/成长', '娱乐/偶像', '爆笑/无厘头', '悬疑/惊悚', '韩国青春文学', '轻小说', '玄幻/新武侠/魔幻/科幻', '大陆原创', '校园', '爱情/情感', '古代言情', '港台青春文学', '其他国外青春文学']
['http://category.dangdang.com/cp01.01.13.00.00.00.html', 'http://category.dangdang.com/cp01.01.11.00.00.00.html', 'http://category.dangdang.com/cp01.01.03.00.00.00.html', 'http://category.dangdang.com/cp01.01.05.00.00.00.html', 'http://category.dangdang.com/cp01.01.06.00.00.00.html', 'http://category.dangdang.com/cp01.01.04.00.00.00.html', 'http://category.dangdang.com/cp01.01.10.00.00.00.html', 'http://category.dangdang.com/cp01.01.15.00.00.00.html', 'http://category.dangdang.com/cp01.01.07.00.00.00.html', 'http://category.dangdang.com/cp01.01.08.00.00.00.html', 'http://category.dangdang.com/cp01.01.01.00.00.00.html', 'http://category.dangdang.com/cp01.01.02.00.00.00.html', 'http://category.dangdang.com/cp01.01.14.00.00.00.html', 'http://category.dangdang.com/cp01.01.09.00.00.00.html', 'http://category.dangdang.com/cp01.01.12.00.00.00.ht

In [5]:
Gclass[0],sclassion[0],sclassurl[0],lensclass[0]

('青春文学', '影视写真', 'http://category.dangdang.com/cp01.01.13.00.00.00.html', 15)

In [24]:
def getbookinfo(Gclass,sclassion,sclassurl,lensclass):################获取图书信息
        pivot=0###########标定位置，方便小分类分段截取
        for i in range(len(Gclass)):
            if Gclass[i]=='韩文原版书':#####去掉空
                continue
            gclass=Gclass[i]            ####大分类49
            sclass=sclassion[pivot:pivot+lensclass[i]]######小分类679
            bookurl=sclassurl[pivot:pivot+lensclass[i]]   #######小分类url-679
            pivot+=lensclass[i] 
            
            #############  1----15-----15
            for _ in range(lensclass[i]):
                url=bookurl[_]
                html = requests.get(url)
                html.encoding = 'gb2312'
                soup = BeautifulSoup(html.text, 'html.parser')########对url进行处理 ，进入分类书籍页面
                book_title=soup.find_all('a',dd_name='单品图片')  #####为获取url集准备
                for i in book_title:
                    try:
                        resp=urlopen(i.get('href'))
                        soup=BeautifulSoup(resp,'html.parser')
                        booktitle=soup.find('h1').get('title')    #####进入具体某本书的页面
                        bookdetail=soup.find('span',class_='head_title_name').get('title')
                        bookdetail=bookdetail[0:-1]
                        bookprice=float(soup.find('p',id='dd-price').text[2:].strip())
                        author=soup.find('a',dd_name='作者').string
                        press=soup.find('a',dd_name='出版社').string
                        publicationtime=soup.find_all('span',class_='t1')[2].text
                        publicationtime=publicationtime[5:-1]
                        bookinfo=[booktitle,publicationtime, author, press,bookprice,bookdetail]
                        writeData(bookinfo,gclass,sclass[_])
                    except:
                        pass
            

In [23]:
html = requests.get('http://category.dangdang.com/cp01.01.13.00.00.00.html')
html.encoding = 'gb2312'
soup = BeautifulSoup(html.text, 'html.parser')########对url进行处理 ，进入分类的具体某本书籍页面
book_title=soup.find_all('a',dd_name='单品图片')
len(book_title)
for i in book_title:
        print(i.get('href'))

http://product.dangdang.com/23823927.html
http://product.dangdang.com/25301853.html
http://product.dangdang.com/24034912.html
http://product.dangdang.com/23990421.html
http://product.dangdang.com/24022104.html
http://product.dangdang.com/24034537.html
http://product.dangdang.com/23410878.html
http://product.dangdang.com/24022140.html
http://product.dangdang.com/23837596.html
http://product.dangdang.com/1295498009.html
http://product.dangdang.com/23349026.html
http://product.dangdang.com/24010638.html
http://product.dangdang.com/23489930.html
http://product.dangdang.com/23966046.html
http://product.dangdang.com/23299202.html
http://product.dangdang.com/1400025387.html
http://product.dangdang.com/1295506309.html
http://product.dangdang.com/1295505609.html
http://product.dangdang.com/1085024580.html
http://product.dangdang.com/23467336.html
http://product.dangdang.com/23990420.html
http://product.dangdang.com/24010641.html
http://product.dangdang.com/1295521609.html
http://product.dangdan

In [14]:
def writeData(bookinfo,Gclass,sclassion):
    path = 'dangdang-book/'+ Gclass+'/' # 设置路径，也可设为相对路径

    # 判断目录是否存在
    if not os.path.exists(path):
        # 目录不存在创建，makedirs可以创建多级目录
        os.makedirs(path)
    try:
        # 保存数据到文件
        with open(path+sclassion+'.csv','a',encoding='utf-8-sig',newline='') as f:
            writer=csv.writer(f,dialect='excel')
            writer.writerow(bookinfo)
        #print('保存成功')
    except Exception as e:
        print('保存失败', e)

In [ ]:
getbookinfo(Gclass,sclassion,sclassurl,lensclass)

保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/穿越/重生/架空.csv'
保存失败 [Errno 2] No such fil

保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/叛逆/成长.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/娱乐/偶像.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang

保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/爆笑/无厘头.csv'
保存失败 [Errno 2] No such file or directo

保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/青春文学/玄幻/新武侠/魔幻/科幻.csv'
保存失败 [Errno 2] No such file or directory

保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功


...

保存失败 [Errno 2] No such file or directory: 'dangdang-book/体育/运动/武术/气功.csv'
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保

保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/布艺/不织布.csv'
保存失败 [Errno 

保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/刺绣/十字绣.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/毛线编织/绳编.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/毛线编织/绳编.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/毛线编织/绳编.csv'
保存失败 [Err

保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/羊毛/串珠/粘土.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/手工/DIY/

保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directory: 'dangdang-book/成功/励志/出国/留学.csv'
保存失败 [Errno 2] No such file or directo